In [8]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys
import nonlinear_QM_FPJPA as Fs
import wx_programs as ex 
import bnc 
from generator import *
from hardware_config import *
from values import * 
from qubit_class import *

In [9]:
pi = np.pi
save_dir = fr"{main_directory}\Python Scripts\Important Blue Fridge Python Files\New\nonlinear_QM\data"
# Example instantiation
q1 = Qubit(
    qubit_id="q1",
    ro_freq=6.7275,
    ge_time=51,
    ef_time=77,
    ge_ssm=-0.111,
    ef_ssm=-0.2525
)

q2 = Qubit(
    qubit_id="q2",
    ro_freq=6.65555,
    ge_time=40,
    ef_time=0,  # Set 0 if not provided in the dictionary
    ge_ssm=-0.152,
    ef_ssm=-0.224
)

print(q1)
print(q2)

Qubit(q1, ro_freq=6.7275, ge_time=51, ef_time=77, ge_ssm=-0.111, ef_ssm=-0.2525)
Qubit(q2, ro_freq=6.65555, ge_time=40, ef_time=0, ge_ssm=-0.152, ef_ssm=-0.224)


In [ ]:
if pis_nopi: #run to correct two qubit rabi tomography
    num_steps = 3
    reps = 15000
    if which_qubit == 0:
        
        Fs.pi_nopi(off = 0,coef=0,pi_ge=pi_ge_time,q = which_qubit,r=which_qubit,ROIF=ROIF2,ssm_ge = ssm_ge); #|g>, keeps population in ground state
        # Fs.pi_nopi(off = 1,coef=1,pi_ge=pi_ge_time,q= which_qubit,r=which_qubit,ROIF=ROIF2,ssm_ge = ssm_ge); #|e>, sends population to |e>
        #Fs.pi_nopi(off = 2,coef=1,coefpief=1,pi_ge=pi_ge_time,pi_ef=pi_ef_time,q= which_qubit,ROIF=ROIF2,ssm_ge = ssm_ge,ssm_ef=ssm_ef);
        
    if which_qubit == 1:
        Fs.pi_nopi(off = 0,coef=0,pi_ge=pi_ge_time,q = which_qubit,r=which_qubit,ROIF=ROIF1,ssm_ge = ssm_ge); #|g>, keeps population in ground state
        Fs.pi_nopi(off = 1,coef=1,pi_ge=pi_ge_time,q= which_qubit,r=which_qubit,ROIF=ROIF1,ssm_ge = ssm_ge); #|e>, sends population to |e>
        #Fs.pi_nopi(off = 2,coef=1,coefpief=1,pi_ge=pi_ge_time,pi_ef=pi_ef_time,q= which_qubit,ROIF=ROIF1,ssm_ge = ssm_ge,ssm_ef=ssm_ef);
